In [1]:
%matplotlib inline

import quandl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from talib import RSI, BBANDS, MACD, AROON
import datetime as dt

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

### Convert tick data to OHLC

In [78]:
# Random tick generator

import datetime as dt
import time 

maxN = 1000

index = pd.date_range(start=dt.datetime.now(), periods=maxN, freq="15s", name="Date")


if False:
    arr = np.zeros(maxN+1, dtype='datetime64[ns]')

    arr[0] = dt.datetime.now()

    for i in np.linspace(1,maxN,maxN, dtype='int'):
        arr[i] = arr[i-1] + np.timedelta64(np.random.randint(40,360), 's' )
    
df5 = pd.DataFrame(data = {"A":np.linspace(1,maxN,maxN)}, index=index)


# Converts pandas frame with tick data to OHLC
'''

df6 = df5.groupby([df5['Date'].dt.hour, df5['Date'].dt.minute])

df7 = pd.DataFrame(
    data = {"date": df6.max()["Date"].apply(lambda x: dt.datetime.replace(x, microsecond=0, second=0)), 
            "high":df6.max()["A"], "low":df6.min()["A"], "open": df6.first()["A"], "close": df6.last()["A"]})
'''

df7 = df5.resample('15min').agg({'A':{'open':'first','high':'max','low':'min','close':'last'}})["A"]
df7.head()

C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



,open,high,low,close
Date,,,,
2019-03-27 12:15:00,1.0,15.0,1.0,15.0
2019-03-27 12:30:00,16.0,75.0,16.0,75.0
2019-03-27 12:45:00,76.0,135.0,76.0,135.0
2019-03-27 13:00:00,136.0,195.0,136.0,195.0
2019-03-27 13:15:00,196.0,255.0,196.0,255.0


### Import data from CSV/Quandl

In [2]:
max_holding = 100

source = "CSV"

symbol = 'NSE/TCS'

In [3]:
if source == "CSV":
    df = pd.read_csv("nse_50.csv")
    df = df.set_index('Date')

    #Convert date from string to date format if required - uncomment below line
    df.index = [dt.datetime.strptime(date, '%Y-%m-%d') for date in df.index]

    price = pd.DataFrame({'open':df["Open"],'high':df["High"],'low':df["Low"],'close':df["Close"],'adjClose':df["Adj Close"],"volume":df["Volume"]})


In [4]:
price.head()

,open,high,low,close,adjClose,volume
2014-03-21,6515.200195,6522.899902,6485.700195,6493.200195,6493.200195,189900.0
2014-03-22,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-24,6510.500000,6591.500000,6510.500000,6583.500000,6583.500000,158400.0
2014-03-25,6550.100098,6595.549805,6544.850098,6589.750000,6589.750000,168100.0
2014-03-26,6615.649902,6627.450195,6580.600098,6601.399902,6601.399902,186600.0


In [5]:
#Clean up and filter data
price = price[price.index> dt.datetime(2018,1,1)]
#price = price.iloc[::-1]
price = price.dropna()
close = price["adjClose"].values

# Machine Learning - Classification model

price.head(50)

#### Technical Indicators Calculations

In [7]:
from talib import RSI, BBANDS, MACD, AROON, STOCH, ATR, OBV, ADOSC
#help(ADOSC)

In [8]:
price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.adjClose, fastperiod=12, slowperiod=26, signalperiod=9)

price['RSI'] = RSI(price.adjClose, timeperiod=14)

price['slowk'], price['slowd'] = STOCH(price.high, price.low, price.close)

price['aroondown'], price['aroonup'] = AROON(price.high, price.low)

price['atr'] = ATR(price.high, price.low, price.close)

price['bbup'], price['bbmid'], price['bblow'] = BBANDS(price.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)

price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)

#### Support and resistance calculation

In [9]:
# Pivot points

PP = (price.high+price.low+price.close)/3
R1 = 2 * PP - price.low
S1 = 2 * PP - price.high

R2 = PP + (price.high - price.low)
S2 = PP - (price.high - price.low)

R3 = price.high + 2 * (PP - price.low)
S3 = price.low - 2 * (price.high - PP)

In [10]:
# Fibonacci Pivot

PP = (price.high+price.low+price.close)/3
S1 = PP - 0.382 * (price.high - price.low)
S2 = PP - 0.618 * (price.high - price.low)
S3 = PP -         (price.high - price.low)

R1 = PP + 0.382 * (price.high - price.low)
R2 = PP + 0.618 * (price.high - price.low)
R3 = PP +         (price.high - price.low)

#### Candle stick pattern recognition

In [12]:
def bbp(price):
    up, mid, low = BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    bbp = (price["adjClose"] - low) / (up - low)
    return bbp

holdings = pd.DataFrame(index=price.index, data={'Holdings': np.array([np.nan] * price.index.shape[0])})

price['BBP'] = bbp(price)

holdings.loc[((price['RSI'] < 30) & (price['BBP'] < 0)), 'Holdings'] = max_holding
holdings.loc[((price['RSI'] > 70) & (price['BBP'] > 1)), 'Holdings'] = 0
holdings.ffill(inplace=True)
holdings.fillna(0, inplace=True)

holdings['Order'] = holdings.diff()
holdings.dropna(inplace=True)

In [60]:
#price[holdings[holdings.Order >0].index]
listBuy = price.loc[holdings[holdings.Order >0].index]
listSell = price.loc[holdings[holdings.Order <0].index]
#holdings[holdings.Order ==0].index.values

In [64]:
#Plotting data
trace0 = go.Ohlc(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Closing Price")


trace61 = go.Scatter(x=listBuy.index, y=listBuy.close, marker = dict(color='green', size=10), mode='markers')
trace62 = go.Scatter(x=listSell.index, y=listSell.close, marker = dict(color='red', size=10), mode='markers')

trace1 = go.Scatter(x=price.index, y=price['adjClose'], mode='lines', line=dict(color='rgb(63, 72, 204)'),name='Closing Price',yaxis="y1")
trace5 = go.Scatter(x=price.index, y=price['bbup'], name='BB_up')
trace7 = go.Scatter(x=price.index, y=price['bblow'], name='BB_low', fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)")

fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, shared_yaxes=False, vertical_spacing=0.05, row_width=[1,2,3])


fig.append_trace(trace0, 1, 1)

fig.append_trace(trace1, 2, 1)
fig.append_trace(trace5, 2, 1)
fig.append_trace(trace7, 2, 1)

fig.append_trace(trace61, 2, 1)
fig.append_trace(trace62, 2, 1)


fig['layout'].update(height=750, title="Stock Analysis for "+symbol)
fig['layout']['xaxis']['rangeslider'].update(thickness=0.05)
#fig['layout']['yaxis1'].update(range=[8000,12000], domain=[0.5,1.0])
#fig['layout']['yaxis'].update(domain=[0.10,0.15])
#fig['layout']['yaxis2'].update(domain=[0.9,1])
#fig['layout'].update(annotations=[dict(text="Buy",x=data1[::-1,0],y=data1[::-1,1])])

iplot(fig, filename="Stock Analysis 2")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]



#### TODO: BUY/SELL decisions

In [10]:
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
price.dropna(inplace=True)

#price.drop(columns=['prevClose'], inplace=True)

#print(price.head(5))

#price['Delta'] = price['prevClose']/price['close'] - 1

labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)

#labels[price['Delta']>0] = S.BUY
#labels[price['Delta']<=0] = S.SELL

#price['Holdings'] = labels

labels[(price['macdhist'] > price['macdhist'].shift(1)) & (price['macdhist'].shift(1) < 0 ) & (price['macdhist'] >= 0 )] = S.BUY
labels[(price['macdhist'] < price['macdhist'].shift(1)) & (price['macdhist'].shift(1) > 0 ) & (price['macdhist'] <= 0 )] = S.SELL

In [11]:
# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

# Plotting Results

In [12]:
# Price vs date curve with green and red dots to indicate buy and sell points

trace0 = go.Scatter(x=buy.index, y=buy.adjClose, marker = dict(color='green', size=8), mode='markers', name="Buy")
trace1 = go.Scatter(x=sell.index, y=sell.adjClose, marker = dict(color='red', size=8), mode='markers', name="Sell")

# Closing Price line chart
trace2 = go.Scatter(x=price.index, y=price.adjClose, marker = dict(color='lightgrey', size=2), mode='lines', name="Close Price", yaxis='y1')


# RSI plots
traceRSI = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI')

# MACD plots
traceMACD = go.Scatter(x=price.index, y=price['macd'], name='MACD', line=dict(color='blue'))
traceMACDHist = go.Bar(x=price.index, y=price['macdhist'], name='MACD Hist', marker=dict(color="lightgrey"))
traceMACDSignal = go.Scatter(x=price.index, y=price['macdsignal'], name='MACD signal', line=dict(color='red'))

# Volume Chart
traceV = go.Bar(x=price.index, y=price.volume, name='Volume', marker=dict(color='lightgrey'),yaxis='y2')

fig = tools.make_subplots(rows=4, cols=1, row_width=[1,2,1,4])

# Price
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

fig.append_trace(traceV,2,1)

# MACD
fig.append_trace(traceMACD, 3, 1)
fig.append_trace(traceMACDHist, 3, 1)
fig.append_trace(traceMACDSignal, 3, 1)

# RSI
fig.append_trace(traceRSI, 4, 1)

# Update layout
fig['layout'].update(height=850, plot_bgcolor='rgba(0,0,0,0)')


fig['layout'].update(yaxis = dict(side='right', domain=[0.50,1]), 
                    xaxis  = dict(visible=False),
                    yaxis2 = dict(side='left', showgrid=False),
                    yaxis3 = dict(side='right'),
                    yaxis4 = dict(side='right'))


                     
iplot(fig, filename="Test")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [42]:
help(pd.DataFrame.aggregate)

Help on function aggregate in module pandas.core.frame:

aggregate(self, func, axis=0, *args, **kwargs)
    Aggregate using one or more operations over the specified axis.
    
    .. versionadded:: 0.20.0
    
    Parameters
    ----------
    func : function, string, dictionary, or list of string/functions
        Function to use for aggregating the data. If a function, must either
        work when passed a DataFrame or when passed to DataFrame.apply. For
        a DataFrame, can pass a dict, if the keys are DataFrame column names.
    
        Accepted combinations are:
    
        - string function name.
        - function.
        - list of functions.
        - dict of column names -> functions (or list of functions).
    
    
    axis : {0 or 'index', 1 or 'columns'}, default 0
        - 0 or 'index': apply function to each column.
        - 1 or 'columns': apply function to each row.
    *args
        Positional arguments to pass to `func`.
    **kwargs
        Keyword argume